# A Guided Tour of Ray Core: Remote Stateless Tasks

Adopted from [Anyscale](https://github.com/anyscale/academy). Under Apache 2.0 License 

<img src="https://github.com/anyscale/academy/blob/dbcac28e08caf83a9d7937f3bcb2747d6f6e67ef/images/ray_basic_patterns.png?raw=1" height="25%" width="50%">

In [ ]:
print('NOTE: Intentionally crashing session to use the newly installed library.\n')

!pip uninstall -y pyarrow
!pip install ray

# A hack to force the runtime to restart, needed to include the above dependencies.
import os
os._exit(0)

NOTE: Intentionally crashing session to use the newly installed library.

Found existing installation: pyarrow 6.0.1
Uninstalling pyarrow-6.0.1:
  Successfully uninstalled pyarrow-6.0.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 53.2 MB 137 kB/s 
     |████████████████████████████████| 144 kB 56.5 MB/s 
     |████████████████████████████████| 4.1 MB 43.7 MB/s 
     |████████████████████████████████| 8.8 MB 29.1 MB/s 
     |████████████████████████████████| 461 kB 63.1 MB/s 
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.46.3
    Uninstalling grpcio-1.46.3:
      Successfully uninstalled grpcio-1.46.3


In [1]:
import os
import time
import logging

import numpy as np
from numpy import loadtxt
import ray

## 1. Tasks Parallel Pattern

Ray converts decorated functions into stateless tasks, scheduled anywhere on a ray worker in the cluster by simply adding the `@ray.remote` decorator. 
All these functions are converted into Ray stateless tasks that will be executed on some worker process on a Ray cluster.

Where they will be executed (and by whom), you don't have to worry about its details. All that is taken care for you. Nor do 
you have to reason about it — all that burden is Ray's job. You simply take your existing Python functions and 
covert them into stateless *Ray Tasks*: as simple as that!

### Example 1: Serial vs Parallelism

Let's look at simple tasks running serially and then in parallel


In [2]:
# A regular Python function.
def regular_function():
    return 1

In [3]:
# A Ray remote function.
@ray.remote
def remote_function():
    return 1

There are a few key differences between the original function and the decorated one:

**Invocation**: The regular version is called with `regular_function()`, whereas the remote version is called with `remote_function.remote()`.

**Return values**: regular_function executes synchronously and returns the result of the function (1), whereas remote_function immediately returns an `ObjectID` (a future) and then executes the task in the background on a separate worker process. The result of the future can be obtained by calling `ray.get` on the `ObjectID`. This is a blocking function.

In [4]:
# Let's invoke the regular function
regular_function()

1

In [5]:
ray.init(
    ignore_reinit_error=True,
    logging_level=logging.ERROR,
)

RayContext(dashboard_url='', python_version='3.7.13', ray_version='1.12.1', ray_commit='4863e33856b54ccf8add5cbe75e41558850a1b75', address_info={'node_ip_address': '172.28.0.2', 'raylet_ip_address': '172.28.0.2', 'redis_address': None, 'object_store_address': '/tmp/ray/session_2022-06-02_06-44-44_632091_135/sockets/plasma_store', 'raylet_socket_name': '/tmp/ray/session_2022-06-02_06-44-44_632091_135/sockets/raylet', 'webui_url': '', 'session_dir': '/tmp/ray/session_2022-06-02_06-44-44_632091_135', 'metrics_export_port': 56037, 'gcs_address': '172.28.0.2:63503', 'address': '172.28.0.2:63503', 'node_id': '536987c85bc72073d23c2574abf6e6fa40eea6de13a42fd3a517f0dc'})

In [6]:
# Let's invoke the remote regular function.
remote_function.remote()

ObjectRef(c8ef45ccd0112571ffffffffffffffffffffffff0100000001000000)

In [7]:
ray.get(remote_function.remote())

1

**No Parallelism**: Invocations of `regular_function` in a loop happen `serially`:

In [8]:
# These are executed one at a time, back-to-back.
result = 0
for _ in range(10):
    result += regular_function()
assert result == 10

**Parallelism**: Invocations of `remote_function` in a loop happen `asynchronously` and in parallel:

In [9]:
# Executing these functions happens at the same time in the background, and we get the results using ray.get.
results = []
for _ in range(10):
    results.append(remote_function.remote())
assert sum(ray.get(results)) == 10

### Example 2: Adding two np arrays

<img src="https://github.com/anyscale/academy/blob/dbcac28e08caf83a9d7937f3bcb2747d6f6e67ef/images/task_api_add_array.png?raw=1" width="60%" height="40%">

Define a function as a Ray task to read an array

In [10]:
@ray.remote
def read_array(fn: str) -> np.array:
    arr = loadtxt(fn, comments="#", delimiter=",", unpack=False)
    return arr.astype('int')

Define a function as a Ray task to add two np arrays return the sum

In [11]:
@ray.remote
def add_array(arr1: np.array, arr2: np.array) -> np.array:
    return np.add(arr1, arr1)

Define a function as a Ray task to sum the contents of an np array

In [12]:
@ray.remote
def sum_array(arr1: np.array) -> int:
    return np.sum(arr1)

Now let's execute our tasks. But first, we have to initialize Ray. For now we will run Ray locally on our laptop, with a single node, including its headnode,
and use all the cores available.

Ray executes immediately and returns an object reference `ObjectRef` as a future. This enables Ray to parallelize tasks and execute them asynchronously.

But first let's initialize ray.

### Read both arrays. 

Use the `func_name.remote(args)` extention to invoke a remote Ray Task

In [42]:
#create data first
import numpy as np
np.savetxt("file_1.txt", np.random.randint((100,100)),delimiter=",")
np.savetxt("file_2.txt", np.random.randint((100,100)),delimiter=",")

In [43]:
obj_ref_arr1 = read_array.remote("file_1.txt")
print(f"array 1: {obj_ref_arr1}")

array 1: ObjectRef(a631fe8d231813bfffffffffffffffffffffffff0100000001000000)


In [44]:
obj_ref_arr2 = read_array.remote("file_2.txt")
print(f"array 2: {obj_ref_arr2}")

array 2: ObjectRef(79cc316456d39201ffffffffffffffffffffffff0100000001000000)


### Add both arrays

Let's add our two arrays by calling the remote method. *Note*: We are sending Ray `ObjectRef` references as arguments.Those will be resolved inline and fetched from owner. Note that node that creates the ref owns them and stores it in its object store. 

Ray scheduler is aware of where these object references reside or who owns them, so it will schedule this remote task on node on the worker process for data locality.

In [45]:
result_obj_ref = add_array.remote(obj_ref_arr1, obj_ref_arr2)
result_obj_ref

ObjectRef(c1464dc5b2308f10ffffffffffffffffffffffff0100000001000000)

### Fetch the result 

This will block untill finished

In [46]:
result = ray.get(result_obj_ref)
print(f"Result: add arr1 + arr2: \n {result}")

Result: add arr1 + arr2: 
 [128 126]


Add the array elements and get the sum
Note we are sending objRefs as arguments to the function
Ray will resolve or fetch the value of these arrays. 

In [47]:
sum_1 = ray.get(sum_array.remote(obj_ref_arr1))
sum_2 = ray.get(sum_array.remote(obj_ref_arr2))

In [48]:
print(f'Sum of arr1: {sum_1}')
print(f'Sum of arr2: {sum_2}')

Sum of arr1: 127
Sum of arr2: 119


### Example 3: Generating Fibonnaci series

Let's define two functions: one runs locally or serially, the other runs on a Ray cluster (local or remote). This example is borrowed and refactored from our blog: [Writing your First Distributed Python Application with Ray](https://www.anyscale.com/blog/writing-your-first-distributed-python-application-with-ray). (This is an excellent tutorial to get started with the concept of why and when to use Ray tasks and Ray Actors. Highly recommended read!)

Another similar blog of interest is how to compute the value of **pi**: [How to scale Python multiprocessing to a cluster with one line of code](https://medium.com/distributed-computing-with-ray/how-to-scale-python-multiprocessing-to-a-cluster-with-one-line-of-code-d19f242f60ff).

In [19]:
# Local execution 
def generate_fibonacci(sequence_size):
    fibonacci = []
    for i in range(0, sequence_size):
        if i < 2:
            fibonacci.append(i)
            continue
        fibonacci.append(fibonacci[i-1]+fibonacci[i-2])
    return len(fibonacci)

In [20]:
# Remote Task with just a wrapper
@ray.remote
def generate_fibonacci_distributed(sequence_size):
    return generate_fibonacci(sequence_size)

In [21]:
# Get the number of cores 
os.cpu_count()

2

In [22]:
# Normal Python in a single process 
def run_local(sequence_size):
    results = [generate_fibonacci(sequence_size) for _ in range(os.cpu_count())]
    return results

In [23]:
%%time
run_local(100000)

CPU times: user 410 ms, sys: 1.15 s, total: 1.56 s
Wall time: 1.57 s


[100000, 100000]

In [24]:
# Distributed on a Ray cluster
def run_remote(sequence_size):
    results = ray.get([generate_fibonacci_distributed.remote(sequence_size) for _ in range(os.cpu_count())])
    return results

In [25]:
%%time
run_remote(100000)

CPU times: user 20.2 ms, sys: 7.67 ms, total: 27.8 ms
Wall time: 1.51 s


[100000, 100000]

### Concept for this Exercise - Task Dependencies

Consider the following basic remote function that returns the argument passed to it. If we pass in some normal Python objects, the results returned by ray.get should be the same objects.

In [26]:
@ray.remote
def f(x):
    return x

x1_id = f.remote(1)
ray.get(x1_id)

1

In [27]:
x2_id = f.remote([1, 2, 3])
ray.get(x2_id)

[1, 2, 3]

However, `object IDs` can also be passed into remote functions. When the function is executed, Ray will automatically substitute the underlying Python object that the object ID refers to. In a sense, it's the same as calling `ray.get` on each argument that's passed in as an argument.

In [28]:
y1_id = f.remote(x1_id)
ray.get(y1_id)

1

In [29]:
y2_id = f.remote(x2_id)
ray.get(y2_id)

[1, 2, 3]

When implementing a remote function, the function should expect a regular Python object regardless of whether the caller passes in a regular Python object or an object ID.

**These task dependencies affect scheduling**. In the example above, the task that creates `y1_id` depends on the task that creates `x1_id`. This means that:

 * The second task will not be executed until the first task has finished executing.
 * If the two tasks are scheduled on different machines, the output of the first task (the value corresponding to x1_id) will be copied over the network to the machine where the second task is scheduled.

### Example 4: Task Dependencies: Aggregating Values Efficiently

Task dependencies can be used in much more sophisticated ways. For example, suppose we wish to aggregate 8 values together. This example uses naive integer addition, but in many applications, aggregating large vectors across multiple machines can be a bottleneck. In this case, changing a single line of code can change the aggregation’s running time from linear to logarithmic in the number of values being aggregated.

<img src="https://github.com/anyscale/academy/blob/dbcac28e08caf83a9d7937f3bcb2747d6f6e67ef/images/task_dependencies_graphs.png?raw=1" height="50%" width="70%">

In [30]:
# define a task to add two intergers
@ray.remote
def add(x, y):
    time.sleep(1)
    return x + y

#### Add values the slow approach

In [31]:
values = [i for i in range(1, 8)]

In [32]:
%%time

while len(values) > 1:
    values = [add.remote(values[0], values[1])] + values[2:]
result = ray.get(values[0])
print(result)

28
CPU times: user 73.1 ms, sys: 15.2 ms, total: 88.3 ms
Wall time: 6.03 s


#### Add values the faster approach

In [33]:
%%time

while len(values) > 1:
    values = values[2:] + [add.remote(values[0], values[1])]
result = ray.get(values[0])
print(result)

28
CPU times: user 256 µs, sys: 1.18 ms, total: 1.44 ms
Wall time: 4.83 ms


### Example 5: Concept of nested parallelism

The goal of this exercise is to show how to create nested tasks by calling a remote function inside of another remote function. Remote functions can call other functions. For example, consider the following.

In [34]:
@ray.remote
def f():
    return 1

@ray.remote
def g():
    # Call f 4 times and return the resulting object IDs.
    results = []
    for _ in range(4):
      results.append(f.remote())
    return results

@ray.remote
def h():
    # Call f 4 times, block until those 4 tasks finish,
    # retrieve the results, and return the values.
    results = []
    for _ in range(4):
      results.append(f.remote())
    return ray.get(results)

Then calling `g` and `h` produces the following behavior.

In [35]:
ray.get(g.remote())

[ObjectRef(ad4e74223c3d479bffffffffffffffffffffffff0100000001000000),
 ObjectRef(a437665df9dd00c0ffffffffffffffffffffffff0100000001000000),
 ObjectRef(c29d5406167e339dffffffffffffffffffffffff0100000001000000),
 ObjectRef(0e554d06ce071dcbffffffffffffffffffffffff0100000001000000)]

In [36]:
ray.get(h.remote())

[1, 1, 1, 1]

In [37]:
# Normally will want to shutdown
ray.shutdown()

---

### References

1. [Modern Parallel and Distributed Python: A Quick Tutorial on Ray](https://towardsdatascience.com/modern-parallel-and-distributed-python-a-quick-tutorial-on-ray-99f8d70369b8) by Robert Nishihara, co-creator of Ray and co-founder Anyscale
2. [Ray Core Introduction](https://www.anyscale.com/events/2022/02/03/introduction-to-ray-core-and-its-ecosystem) by Jules S. Damji